In [1]:
import glob
import skimage.io
import os
import numpy as np
import tifffile

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt

# import user classes
from model.AttentionUnetResnet import AttentionUnetResnet50

%load_ext autoreload
%autoreload 2

In [ ]:
folder_list = glob.glob(r'prediction/**')

for folder in folder_list[:]:
    image_list =  glob.glob(os.path.join(folder, r"**/*_resliced.tif")) 
    for image_path in image_list[:]:
        mm = tifffile.memmap(image_path)
        image = np.array(mm[100,...])

        model_dir = r'networks/Attention_Unet_Resnet50-20220425T0252'

        model = AttentionUnetResnet50(model_dir = model_dir,
                                        visible_gpu = 0,
                                        use_cpu = False,
                                        config_filepath=None,
                                        for_prediction=True,
                                        patch_size = list(image.shape),
                                        pad_image=True,
                                        use_tiling = False,
                                        overlap_size = [64,64],
                                        untiling_method = "max_blend")

        model.create_prediction_model(os.path.join(model_dir,'model-200000.index'))
        output_folder = os.path.split(image_path)[0]
        test = model.predict_batch_from_directory(output_folder, image_ext='*_resliced.tif', suffix='-preds-200000')